In [84]:
entities1 = ["vodafon","türkcell", "kick","twitch"] 

sentence1 = "vodafon güzel. türkcell kötü. kick iyi. twitch mükkemmel."

def find_sentence_counter(sentencetext, entities):
    countermap = dict()
    if sentencetext[-1] == ".": #remove the last dot(.) to get the correct sentence length
        lt = len(sentencetext)
        sentencetext = sentencetext[0:lt-1]
    sentencelist = sentencetext.split(".") # get the sentence list
    index = 0
    count = 0
    for sentence in sentencelist: # for each sentence in sentence list, calculate the aspect number in the sentence
        sentence_words = sentence.split(" ") # get the words
        
        for words in sentence_words:
            if words in entities:
                count = count +1
                countermap[index] = count
            
        index= index +1
        count=0 
    return countermap






mydict=find_sentence_counter(sentence1,entities1)


result = False


if all(mydict.values()) == 1:
    result=True


print(result)







True


In [85]:
import spacy

nlp = spacy.load('tr_core_news_trf')


fiilimsi_suffixes = [
    "ken", "alı", "eli", "madan", "meden", "ince", "ınca", "unca", "ünce",
    "ıp", "up", "üp", "arak", "erek", "dıkça", "dikçe", "dukça", "dükçe",
    "tıkça", "tikçe", "tukça", "tükçe", "a", "e", "r", "maz", "mez", 
    "casına", "cesine", "meksizin", "maksızın", "dığında", "diğinde", 
    "duğunda", "düğünde", "tığında", "tiğinde", "tuğunda", "tüğünde"
]

def find_fiilimsi(sentence):
    lengthofsentence = len(sentence)
    doc = nlp(sentence)
    
    fiilimsis = []
    noun_positions = []
    
    for token in doc:
    
        if any(token.text.endswith(suffix) for suffix in fiilimsi_suffixes):
            
            if token.idx != len(sentence) - len(token.text) :
                head_token = token.head
               # print(f" {token.dep_}")
                if token.dep_ == "advcl" and (head_token.pos_ == 'VERB' or head_token.pos_ =='ADJ') :
                    
                 # 84 -9 = 75
                    fiilimsis.append(token.text)

                

            
        elif token.pos_ == "NOUN":
            noun_positions.append((token.text, token.idx))
    
    

    return fiilimsis

sentence = "Vodafone kullanmadan yoruldum."
res=find_fiilimsi(sentence)
res

C:\Users\HAMZA\AppData\Roaming\Python\Python310\site-packages\spacy_transformers\layers\hf_shim.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._model.load_state_

['kullanmadan']

In [92]:
entities1 = ["Vodafone","Turkcell"]

sentence1 = "Vodafone Turkcell güzelken çok pahalılar"
example5 = "Twitch kick güzelken whatsapp çalışmıyor" # 
ent5 = ["Twitch","kick","whatsapp"]
sentence2 = ""
def howmanyentity(sentence, entity):
    count = 0

    words = sentence.split(" ")
    for i in words:
        if i in entity:
            count= count+1
            

    return count

howmanyentity(example5,ent5)




    

3

In [87]:
def usecases(mydict,sentencetext,entities): # mydict contains dict of  sentences id  and their entity values.
   
    print(len(entities))
    print(len(find_fiilimsi(sentencetext)))
    if(len(entities)==1 and ("." not in sentencetext) and (len(find_fiilimsi(sentencetext)) ==1  or (len(find_fiilimsi(sentencetext))) == 0)):
        return sentencetext ## assuming there is 1 entity and 1 sentence


    if len(entities) > len(sentencetext.split(".")):
        
        resultofcomplex = []
        sentences = sentencetext.split(".")
        # check each sentence for entity number
        
        for sentence in sentences:
            if howmanyentity(sentence,entities) > 1:
                # that means in this sentence there is more than 1 entitiy so we should make 2 sentences(if we assume sentence contains 1 fiilimsi and 2 sentences)
                # find the fiilimsi and their number if it is 1 make it two sentences
                # while doing it check the entitiy places if two of the entity places before the fiilimsi entity fiilimsi rest of the sentence
                # if the fiilimsi is between entities, split it right on the fiilimsi 
                print(sentence)

                myentities = []
                for i in entities:
                    if i in sentence:
                        myentities.append(i)
                
                fiilimsiler = find_fiilimsi(sentence) # lets consider there is one 2 entity and one fiilimsi
                
                
                positionfiilimsi = sentence.find(fiilimsiler[0])
                positionentity1 = sentence.find(myentities[0]) # find corrresponding entities and put them in here
                positionentity2 = sentence.find(myentities[1])
                
                

                if(positionentity1 < positionfiilimsi  and positionentity2 < positionfiilimsi):
                    # 2 entitiy de fiilimsiden önce
                    if(positionentity1<positionentity2):
                        
                        
                        if positionentity1 ==0: 
                            sentence1  = sentence[positionentity1:positionentity2] + sentence[positionfiilimsi:]
                            sentence2 = sentence[positionentity2:positionfiilimsi] + sentence[positionfiilimsi:]
                            resultofcomplex.append(sentence1)
                            resultofcomplex.append(sentence2)


                        else:
                            sentence1  = sentence[0:positionentity1] +sentence[positionentity1]+ sentence[positionfiilimsi:]
                            sentence2 = sentence[positionentity2:positionfiilimsi] + sentence[positionfiilimsi:]
                            resultofcomplex.append(sentence1)
                            resultofcomplex.append(sentence2)
                    else:
                        if positionentity2 == 0 :

                            sentence1 = sentence[positionentity2] + sentence[positionfiilimsi]
                            sentence2 = sentence[positionentity1:positionfiilimsi] + sentence[positionfiilimsi]
                            resultofcomplex.append(sentence1)
                            resultofcomplex.append(sentence2)
                        else :
                            sentence1 = sentence[0:positionentity2]+sentence[positionentity2] + sentence[positionfiilimsi]
                            sentence2 = sentence[positionentity1:positionfiilimsi] + sentence[positionfiilimsi]
                            resultofcomplex.append(sentence1)
                            resultofcomplex.append(sentence2)
    
                elif ( positionentity1 < positionfiilimsi and positionentity2 > positionfiilimsi):
                    
                    sentence1 = sentence[positionentity1:(positionfiilimsi + len(fiilimsiler[0]))]
                    sentence2 = sentence[(positionfiilimsi + len(fiilimsiler[0])):]
                    resultofcomplex.append(sentence1)
                    resultofcomplex.append(sentence2)
            
                elif(positionentity2 < positionfiilimsi and positionfiilimsi < positionentity1):
                
                    sentence1 = sentence[positionentity2:(positionfiilimsi + len(fiilimsiler[0]))]
                    sentence2 = sentence[(positionfiilimsi + len(fiilimsiler[0])):]
                    resultofcomplex.append(sentence1)
                    resultofcomplex.appen(sentence2)
                
                return resultofcomplex
                
            # there is only one entity in whole sentence so we simply return it as it is.
                


                
        
       
        

        # bağlaç var fiilimsi yok
        # fiilimsi var bağlaç yok
        # bağlaç ve use case var
        # sonunda vodafone var
# 3 aspect ile olanı yap ve bitir.
example = "Vodafone güzelken Turkcell çok yavaş kalması can sıkıcı"
example2 = "Vodafone kick izlerken donmalar oluyor"
example3 = "Vodafone kullanmadan yoruldum"
example4 = "Vodafone memnun değilim"
entities = ["Vodafone"]
dictinoray = find_sentence_counter(example,entities)
result = usecases(sentencetext=example4,entities=["Vodafone"],mydict=dictinoray)
result
        
            
 



1
0


'Vodafone memnun değilim'

In [169]:
def splitting(entity : list, sentence: str):

    sentenceresults = list()

    if sentence.endswith("."):
        sentence = sentence[0:-1]

    
    numberofentities=howmanyentity(sentence=sentence,entity=entity) #change this function to handle empty spaces in the end
    print(f"numberofentities of this function is : {numberofentities}") 
    if ( numberofentities == 1): # if there is only 1 entity
        sentenceresults.append(sentence)

    elif(numberofentities >1 ):
        numberoffiilimsi = len(find_fiilimsi(sentence=sentence))
        print(f"Number of fiilimsi is :{numberoffiilimsi}")

        if(numberoffiilimsi==1) :
            print("Hamza") 
            entitiypositions = []
            for i in entity:
                entitiypositions.append(sentence.index(i))

            fiilimsiposition = sentence.find(find_fiilimsi(sentence=sentence)[0])
            print("Emin")
            whereisfiilimsi = all(index < fiilimsiposition for index in entitiypositions)
            if whereisfiilimsi:
            
                
                for i in entitiypositions:
                    entitylen = len(sentence[i:].split()[0])
                    print(f" Empty len is {entitylen}")
                    print(f"entitiyposition: {entitylen}, fiilimsipositions: {fiilimsiposition}")
                    if i == 0:
                        temp_sentence = sentence[:entitylen] + " " + sentence[fiilimsiposition:]
                    if i !=0:
                        temp_sentence = sentence[i:i+entitylen]+ " " + sentence[fiilimsiposition:]
                    sentenceresults.append(temp_sentence)
                    

            else : # that means there is a fiilimsi between entities. there can be entity fiilimsi entity , entity entity fiilimsi entity
                beforeentitiesposition = [] # BEFORE THE FİİLİMSİ ENTİTİES
                afterentitiesposition = [] # AFTER THE FİİLİMSİ ENTİTİES
                fiilimsiposition = sentence.find(find_fiilimsi(sentence=sentence)[0]) # FİİLİMSİ POİZİTON 

                for i in entity:
                    if sentence.find(i) < fiilimsiposition:
                        beforeentitiesposition.append(sentence.find(i))
                    elif sentence.find(i) > fiilimsiposition:
                        afterentitiesposition.append(sentence.find(i))
                
                #handle before entities
                #handle after entities
                #append the result
                for positions in beforeentitiesposition: # ["Twitch kick güzelken whatsapp çalışmıyor"] ["Twitch: 0 kick : 7 whatsapp :20"]["güzelken=12"]
                    print("--------------------")
                    
                    entitylen = len(sentence[fiilimsiposition:].split()[0]) # güzelken = 12 güzelken len = 8
                    subsentence = sentence[:fiilimsiposition+entitylen]   # sentence[:20] whatsapp dan öncesi
                    tempb = subsentence
                    for words in entity: #sentence is Twitch kick güzelken beforeentitiesposition is [twitch pozisyonu kick pozisyonu]
                        if subsentence.find(words) == positions and len(beforeentitiesposition)!=1 : #  0 ve 7 eğer 20 eğer eşit değilse  0 7 20 yani 20 alıcak
                            tempb=tempb.replace(words,"") # entity list contains Twitch kick turkiye. 
                    sentenceresults.append(tempb)
                    
                

                for positions in afterentitiesposition: 
                    print("////////////////////")
                   
                    entitylen = len(sentence[fiilimsiposition:].split()[0])
                    subsentence = sentence[fiilimsiposition+entitylen:]
                    tempa = subsentence
                    for words in entity: # küçük bir hata var
                        if subsentence.find(words) == positions : # with last part after the and  if the entity  is less than the position
                            tempa = tempa.replace(words,"")
                    sentenceresults.append(tempa)
                    
        

    return sentenceresults

#splitting(["Vodafone","Turkcell"],"Vodafone Turkcell güzelken çok pahalılar") # that works
entities3= ["Vodafone","Turkcell"]
example3 = "Vodafone Turkcell güzelken çok pahalılar"  # correct  # 
example16 = "Vodafone güzelken Turkcell çok yavaş kalması can sıkıcı"#not correct # correct -if ı change beforepostion from > to <this one comes uncorrect
example5 = "Twitch kick güzelken whatsapp çalışmıyor" # correct #  correct if ı change beforeposition from > to < this one comes uncorrect
ent5 = ["Twitch","kick","whatsapp"]

splitting(entity=ent5,sentence= example5)



numberofentities of this function is : 3


c:\Python310\lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Number of fiilimsi is :1
Hamza
Emin
--------------------
--------------------
////////////////////


[' kick güzelken', 'Twitch  güzelken', ' whatsapp çalışmıyor']

In [171]:
splitting(entity=entities3,sentence= example3)

numberofentities of this function is : 2
Number of fiilimsi is :1
Hamza
Emin
 Empty len is 8
entitiyposition: 8, fiilimsipositions: 18
 Empty len is 8
entitiyposition: 8, fiilimsipositions: 18


['Vodafone güzelken çok pahalılar', 'Turkcell güzelken çok pahalılar']

In [170]:
splitting(entity=entities3,sentence= example16)

numberofentities of this function is : 2
Number of fiilimsi is :1
Hamza
Emin
--------------------
////////////////////


['Vodafone güzelken', ' Turkcell çok yavaş kalması can sıkıcı']

In [172]:
splitting(entity=ent5,sentence= example5)

numberofentities of this function is : 3
Number of fiilimsi is :1
Hamza
Emin
--------------------
--------------------
////////////////////


[' kick güzelken', 'Twitch  güzelken', ' whatsapp çalışmıyor']

8